# Alpha Diversity Analysis

## Import Libraries

In [1]:
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization

%matplotlib inline

In [2]:
data_dir = 'data'
data_dir_phyl = 'data/phylogeny'
data_dir_div = 'data/diversity'

## Alpha Rarefaction

To perform rarefaction, we first need to decide which sampling depth is best suited for our dataset. For this, we will analyse how sampling depth impacts within-sample diversity estimates (= alpha diversity) with the alpha-rarefaction action. This action generates interactive alpha rarefaction curves for sequencing depths between min_depth and max_depth and computes 10 (default) rarefied tables with corresponding alpha diversity metrics at each sampling depth step.

In [3]:
! qiime diversity alpha-rarefaction \
    --i-table $data_dir/dada/dada2_table.qza \
    --i-phylogeny $data_dir_phyl/fasttree_tree_rooted.qza \
    --p-max-depth 10000 \
    --m-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir_div/alpha_rarefaction.qzv

^C

Aborted!


In [4]:
Visualization.load(f"{data_dir_div}/alpha_rarefaction.qzv")

<visualization: Visualization uuid: 6f523f6b-876e-4e37-ad28-717ceb6b7649>

## Diversity Analysis

In [5]:
#if error occurs we need to delete old core_metrics_results directory so it can create a new directory 
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/dada/dada2_table.qza \
  --i-phylogeny $data_dir_phyl/fasttree_tree_rooted.qza \
  --m-metadata-file $data_dir/pundemic_metadata.tsv \
  --p-sampling-depth 1500 \
  --output-dir $data_dir_div/core_metrics_results

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/q2cli/util.py", line 273, in get_plugin_manager
    return qiime2.sdk.PluginManager.reuse_existing()
  File "/opt/conda/lib/python3.9/site-packages/qiime2/sdk/plugin_manager.py", line 58, in reuse_existing
    raise UninitializedPluginManagerError
qiime2.sdk.plugin_manager.UninitializedPluginManagerError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/bin/qiime", line 11, in <module>
    sys.exit(qiime())
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.9/site-packages/click/core

Test the associations between categorical metadata columns and the Faith Phylogenetic Diversity (a measure of community richness) metric using the qiime diversity alpha-group-significance method (implementation of a one-way ANOVA method, namely Kruskal-Wallis test):

In [6]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir_div/core_metrics_results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata.tsv \
  --o-visualization $data_dir_div/core_metrics_results/faith_pd_group_significance.qzv

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/q2cli/util.py", line 273, in get_plugin_manager
    return qiime2.sdk.PluginManager.reuse_existing()
  File "/opt/conda/lib/python3.9/site-packages/qiime2/sdk/plugin_manager.py", line 58, in reuse_existing
    raise UninitializedPluginManagerError
qiime2.sdk.plugin_manager.UninitializedPluginManagerError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/bin/qiime", line 11, in <module>
    sys.exit(qiime())
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.9/site-packages/click/core

In [7]:
Visualization.load(f"{data_dir_div}/core_metrics_results/faith_pd_group_significance.qzv")

<visualization: Visualization uuid: 587086e3-04d8-458e-b7b7-5bf9df772b6d>

Next, we will test whether numeric sample metadata columns are correlated with microbial community richness by using the qiime diversity alpha-correlation method (implementation of Spearman correlation):

In [8]:
#make metadata columns numeric
metadata = pd.read_csv(f"{data_dir}/pundemic_metadata.tsv", sep='\t')
metadata.dtypes

id                               object
patient_id                       object
age                              object
sex                              object
ethnicity                        object
continent                        object
country                          object
region                           object
city                             object
group                            object
disease_subgroup                 object
blinded_clinical_response        object
puns_per_hour_pre_treatment     float64
puns_per_hour_post_treatment    float64
time_point                       object
dtype: object

In [9]:
metadata['age'] = pd.to_numeric(metadata['age'], errors = 'coerce')
metadata['patient_id'] = pd.to_numeric(metadata['patient_id'], errors = 'coerce')
metadata.dtypes

id                               object
patient_id                      float64
age                             float64
sex                              object
ethnicity                        object
continent                        object
country                          object
region                           object
city                             object
group                            object
disease_subgroup                 object
blinded_clinical_response        object
puns_per_hour_pre_treatment     float64
puns_per_hour_post_treatment    float64
time_point                       object
dtype: object

In [10]:
metadata.to_csv(f'{data_dir}/pundemic_metadata_numeric.tsv', sep='\t', index=False)

In [11]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir_div/core_metrics_results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/pundemic_metadata_numeric.tsv \
  --o-visualization  $data_dir_div/core_metrics_results/faith_pd_group_significance_numeric.qzv

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
Saved Visualization to: data/diversity/core_metrics_results/faith_pd_group_significance_numeric.qzv


In [12]:
Visualization.load(f"{data_dir_div}/core_metrics_results/faith_pd_group_significance_numeric.qzv")

<visualization: Visualization uuid: 05a2ca27-ac0c-4932-9d8f-322251a54f8f>